In [2]:
import gurobipy as gp
from gurobipy import Model, GRB

# Given Tasks:
# T1 = (6, 2)  # Task period = 6, execution time = 2
# T2 = (12, 3) # Task period = 12, execution time = 3
# T3 = (18, 4) # Task period = 18, execution time = 4

In [ ]:
# Hyperperiod
hyperperiod = 36

# Set of jobs, e.g., J12 is the second job of the first task.
jobs = ["J11", "J12", "J13", "J14", "J15", "J16", 
        "J21", "J22", "J23", 
        "J31", "J32"]

# Release times for jobs
releasetimes = {
    "J11": 0, "J12": 6, "J13": 12, "J14": 18, "J15": 24, "J16": 30,
    "J21": 0, "J22": 12, "J23": 24,
    "J31": 0, "J32": 18
}

# Deadlines for jobs
deadlines = {
    "J11": 6, "J12": 12, "J13": 18, "J14": 24, "J15": 30, "J16": 36,
    "J21": 12, "J22": 24, "J23": 36,
    "J31": 18, "J32": 36
}

# Execution times for jobs
executiontimes = {
    "J11": 2, "J12": 2, "J13": 2, "J14": 2, "J15": 2, "J16": 2,
    "J21": 3, "J22": 3, "J23": 3,
    "J31": 4, "J32": 4
}

# Create Gurobi model
m = Model()

# Variables for job start times
x = m.addVars(jobs, name="x")

# Constraint: Start time of each job must be after its release time
for job in jobs:
    m.addConstr(x[job] >= releasetimes[job], name=f"release_time_{job}")

# Constraint: End of job must be before its deadline
for job in jobs:
    m.addConstr(x[job] + executiontimes[job] <= deadlines[job], name=f"deadline_{job}")


# Add constraints to ensure no two jobs from different tasks overlap
# A large constant (Big-M), must be greater than the hyperperiod
M = hyperperiod + 1
for job1 in jobs:
    for job2 in jobs:
        if job1 != job2 and job1[1] != job2[1]:  # Ensure jobs are from different tasks
            # Create a binary variable to decide the order of jobs
            h = m.addVar(vtype=GRB.BINARY, name=f"h_{job1}_{job2}")
            
            # Enforce non-overlap using Big-M constraints
            m.addConstr(x[job1] + executiontimes[job1] <= x[job2] + M * (1 - h), 
                        name=f"job1_before_job2_{job1}_{job2}")
            m.addConstr(x[job2] + executiontimes[job2] <= x[job1] + M * h, 
                        name=f"job2_before_job1_{job1}_{job2}")


# Disable presolve
m.setParam("Presolve", 0)

# Optimize the model
m.optimize()

Set parameter Username
Set parameter LicenseID to value 2606738
Academic license - for non-commercial use only - expires 2026-01-06
Set parameter Presolve to value 0
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-11370H @ 3.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Non-default parameters:
Presolve  0

Optimize a model with 166 rows, 83 columns and 454 nonzeros
Model fingerprint: 0x69836673
Variable types: 11 continuous, 72 integer (72 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+01]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 4e+01]
Found heuristic solution: objective 0.0000000

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 0 

Optimal solution found (tolerance 1.00e-04)


In [4]:
# Output the results
if m.status == GRB.OPTIMAL:
    print("Optimal solution found!")
    for job in jobs:
        print(f"Start time for job {job}: {x[job].X}")
else:
    print("No optimal solution found.")

Optimal solution found!
Start time for job J11: 3.0
Start time for job J12: 9.0
Start time for job J13: 15.0
Start time for job J14: 18.0
Start time for job J15: 27.0
Start time for job J16: 30.0
Start time for job J21: 0.0
Start time for job J22: 12.0
Start time for job J23: 24.0
Start time for job J31: 5.0
Start time for job J32: 20.0
